In [44]:
import pandas as pd
import numpy as np
import re
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from category_encoders import TargetEncoder
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error 
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_score, GridSearchCV
from xgboost import XGBRegressor
from scipy.stats import randint, uniform
from sklearn.utils import resample

In [2]:
data = pd.read_csv('train_update.csv')
data.shape

(788, 26)

# PART 1

<div dir="rtl">
השתמשנו ב api של gemini לצורך חילוץ מידע מעמודת התיאור. במקרה של ערך חסר - השלמנו, ובמקרה של סתירה - החלפנו את הערך כאשר הנתון המקורי היה בעייתי (חריג או לא הגיוני).
<br/>
<br/>
1. סוג הנכס - איחוד קטגוריות שהיו מאותו סוג אך כתובות שונה. הסרנו שורות שאינן דירת מגורים. 
<br/>
2. שכונה - יצרנו רשימת שכונות שניתנו במטלה, והסרנו שורות שלא היו המשכונות הללו. הבנו שיש שורות שנכרו בטעות מפרסומות שהיו בעמוד, ולכן היו בדאטה דירות לא מתל אביב או לא להשכרה. 
<br/>
3. כתובת - החלטנו להסיר את העמודה, כיון שהיא לא קטגורית ולא מספרית, יש בה יותר מידי ערכים מכדי לקודד אותה, והמידע בה מגולם בעמודות שכונה, מרחק מהמרכז ועמודות נוספות שקשורות למיקום. 
<br/>
4. מספר חדרים - החלפנו מספר חדרים 0 (לא הגיוני) לnan כדי שיושלם בשלב הבא בו מילאנו ערכים חסרים בעמודה הזו. 
<br/>
5. קומה וסך קומות - מצאנו כמה טעויות בעמודות הללו: 
-שורות שנשארו בפורמט של מספר מתוך מספר 
-שורות שמספר הקומה בהן היה מורכב מהמספר של הקומה והקומות יחד (גילינו את זה כשבדקנו שורות בהן הקומה גדולה מסך הקומות) 
-שורות בהן המילה קרקע לא הומרה ל0, אלא קיבלה null. והחולפו בהן הערכים בעמודת קומה וסך קומות.
בנינו פונקציות לטיפול בבעיות הללו. 
<br/>
6. שטח - ביצענו בדיקת חריגים. החלטנו להסיר שורות מתחת ל15 מ"ר כיון שמצאנו שמתחת שטח זה רוב השורות אינן דירות. שורות עם שטח גדול מ350 החלפנו לnan כדי שיוחלפו בהמשך.
<br/>
7. שטח גינה - השלמנו ערכים חסרים ב0, כיון שמדובר בדירות ללא גינה לרוב 
<br/>
8. מספר ימים לכניסה - הנחנו שכאשר לא צוין תאריך סביר שהכניסה מיידית, לכן הושלם בשורות החסרות 0. ראינו שזו עמודה עם שונות נמוכה ולכן הפכנו אותה לעמודה בינאריות כאשר 0 מייצג כניסת מיידית, ו-1 כניסה לא מיידית. 
<br/>
9. מספר תשלומים - ראינו שזו עמודה עם שונות נמוכה. השלמנו בערכים החסרים 12, כיון שזה התשלום הנפוץ והסטנדרטי (חודשי). 
<br/>
10. ארנונה חודשית - הפכנו ערכים חריגים ואפסים (שאינם הגיוניים) לnan כדי שיוחלפו בשלב הבא בו טיפלנו בערכים חסרים בעמודה הזו.
<br/>
11. ועד בניין - כאשר סך הקומות היה 0 או 1 הנחנו שלא מדובר בבניין ולכן בערך חסר במקרה כזה השלמנו 0. ערכים חריגים הפכנו לnan כדי שיוחלפו בשלב הבא בו טיפלנו בערכים חסרים בעמודה הזו. 
<br/>
12. עמודות בינאריות - אין ערכים חסרים ולכן הנחנו שהן נלקחו כמו שצריך מהאתר
<br/>
13. מספר תמונות - זהו ערך שלא מתבסס רעיונית על ערכים אחרים ולכן השלמנו בערכים החסרים 0. 
<br/>
14. מרחק מהמרכז - ראינו שחלק מהעמודות היו עם מרחק במטרים וחלקן בק"מ, מצאנו ערך סף להבדל בניהם והמרנו את כל המטרים 
לק"מ. לאחר מכן, השלמנו ערכים חסרים או חריגים באמצעות בדיקה חוזרת עם google api כפי שעשינו בחלק א של הפרויקט. 
<br/>
15. מחיר - הסרנו שורות עם מחיר לא הגיוני (נמוך או גבוה) או מחיר שאינו תקין בבירור אלא סתם הקלדה של מספר כגון 111, 123.
<br/>
<br/>
**בכל מקום שיש ערך סף בו החלטנו שמעליו/מתחתיו נחשב ערך חריג אזי הוא מבוסס על בדיקות של התפלגויות או במקרים מסוימים גם הבנה בסיסת של נדל"ן**

<div dir="rtl">
הוספת פיצ'רים: 
<br/>
בחרנו להוסיף פיצרים שחשבנו שיוכלו להוסיף מידע שעשוי להשפיע על מחיר הדירה. 
<br/>
1. דירוג סוציו-אקונומי לשכונה - יצרנו מילון של דירוג סוציו אקונומי לשכונות לפי מידע של עיריית תל אביב. התאמנו לכל שורה את הדירוג המתאים לה.   
<br/>
2. חילקנו את השכונות לרובעים בתל אביב והתאמנו לכל שורה את הרובע שלה
<br/>
3. קומה / סך קומות - מייצג את המיקום של הדירה בבניין (לבדוק אם התייחסנו לחלוקה ב0)  
<br/>
4. שטח / מספר חדרים - מייצג כמה הדירה מרווחת 
<br/>
5. כדי להקל על האלסטיקנט ללמוד יצרנו עמודות שמייצגות אינטראקציות בין פיצ'רים  

## Initial Data Cleaning

In [3]:
## ירוץ רק על הטריין

def remove_suspect_prices(df):
    """
    מסירה שורות בהן המחיר:
    - הוא אחד מהערכים: 1, 123, 1234
    - מכיל רצף של לפחות 3 ספרות זהות בין 1 ל־8
    """
    suspect_prices = [1, 123, 1234]
    mask_exact = df['price'].isin(suspect_prices)
    mask_pattern = df['price'].astype(str).str.contains(r'([1-8])\1{2,}')
    mask = mask_exact | mask_pattern
    df.drop(index=df[mask].index, inplace=True)

def remove_extreme_and_na_prices(df):
    """
    מסירה שורות עם מחירים חריגים:
    - מחיר גבוה מ־50,000
    - מחיר נמוך מ־2,000
    """
    df.drop(df[(df['price'] > 50000) | (df['price'] < 2000)].index, inplace=True)
    df.dropna(subset=['price'], inplace=True)


In [4]:
#הפעלת הפונקציות על הדאטה
remove_suspect_prices(data)
remove_extreme_and_na_prices(data)

C:\Users\אליאור\AppData\Local\Temp\ipykernel_25500\430222772.py:11: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_pattern = df['price'].astype(str).str.contains(r'([1-8])\1{2,}')


## Functions for Prepare Data

### Parameters

In [5]:
#רשימת שכונות 
neighborhoods = [
    "הצפון הישן החלק המרכזי", "בית שטראוס", "התקווה",
    "לב תל אביב החלק הצפוני", "נוה ברבור",
    "הצפון החדש החלק הדרומי", "נוה אביבים",
    "הצפון הישן החלק הצפוני", "מע\"ר צפוני",
    "הצפון הישן החלק הדרום מזרחי", "רמת אביב ג",
    "פלורנטין", "עג'מי",
    "הקריה", "שבזי",
    "הצפון החדש סביבת ככר המדינה", "שיכון בבלי",
    "הצפון הישן החלק הדרום מערבי", "מונטיפיורי",
    "הגוש הגדול", "מרכז יפו מערבית לשדרות ירושלים",
    "קרית שלום", "נוה שרת", "יפו ד",
    "לב תל אביב החלק הדרומי", "נוה חן", "תל חיים",
    "כרם התימנים", "תל כביר", "כפיר",
    "שפירא", "בצרון", "ניר אביב",
    "הצפון החדש החלק הצפוני", "נוה אליעזר", "תל ברוך צפון",
    "צפון יפו", "נחלת יצחק", "מרכז יפו מזרחית לשדרות ירושלים",
    "נוה שאנן", "גבעת הרצל", "רמת החייל",
    "רמת אביב החדשה", "עזרא",
    "יד אליהו", "רביבים",
    "כוכב הצפון", "גני צהלה",
    "הדר יוסף", "לבנה", "נאות אפקה ב", "אזורי חן",
    "חוף הצוק", "לינקולן",
    "פארק צמרת", "נאות אפקה א",
    "תוכנית ל",
    "צהלון", "המשתלה", "שיכוני חסכון", "אפקה", "דקר", "רמת אביב", "יפו ג", "רמת ישראל", "רמת הטייסים", "נמל תל אביב", "צהלה"]

In [6]:
#מילון רובעים
tel_aviv_districts = {
    "רובע 1": [
        "נוה אביבים",
        "רמת אביב ג",
        "רמת אביב החדשה",
        "כוכב הצפון",
        "אפקה",
        "רמת אביב",
        "הגוש הגדול",
        "אזורי חן",
        "חוף הצוק",
        "תוכנית ל"
    ],
    "רובע 2": [
        "נוה שרת",
        "רמת החייל",
        "רביבים",
        "גני צהלה",
        "הדר יוסף",
        "נאות אפקה ב",
        "נאות אפקה א",
        "המשתלה",
        "צהלה",
        "תל ברוך צפון"
    ],
    "רובע 3": [
        "הצפון הישן החלק המרכזי",
        "הצפון הישן החלק הצפוני",
        "הצפון הישן החלק הדרום מזרחי",
        "הצפון הישן החלק הדרום מערבי",
        "נמל תל אביב"
    ],
    "רובע 4": [
        "שיכון בבלי",
        "פארק צמרת",
        "הצפון החדש החלק הדרומי",
        "הצפון החדש סביבת ככר המדינה",
        "הצפון החדש החלק הצפוני"
    ],
    "רובע 5": [
        "לב תל אביב החלק הצפוני",
        "כרם התימנים",
        "לב תל אביב החלק הדרומי",
        "שבזי",
        "בית שטראוס"
    ],
    "רובע 6": [
        "מונטיפיורי",
        "מע\"ר צפוני",
        "הקריה"
    ],
    "רובע 7": [
        "עג'מי",
        "יפו ד",
        "צהלון",
        "שיכוני חסכון",
        "יפו ג",
        "צפון יפו",
        "מרכז יפו מערבית לשדרות ירושלים",
        "מרכז יפו מזרחית לשדרות ירושלים"
    ],
    "רובע 8": [
        "פלורנטין",
        "קרית שלום",
        "שפירא",
        "נוה שאנן",
        "תל כביר",
        "גבעת הרצל",
        "לינקולן"
    ],
    "רובע 9": [
        "התקווה",
        "נוה ברבור",
        "נוה חן",
        "תל חיים",
        "כפיר",
        "בצרון",
        "ניר אביב",
        "נוה אליעזר",
        "נחלת יצחק",
        "יד אליהו",
        "לבנה",
        "רמת ישראל",
        "רמת הטייסים",
        "עזרא",
        "דקר"
    ],
}

In [7]:
#מילון דירוג סוציו-אקונומי
se_groups = {
    10: [
        'צמרות איילון' ,'אפקה', 'קריית שאול', 'המשתלה', 'צהלה', 'רביבים', 'מע\"ר צפוני', 'פארק צמרת'   
    ],
    9: [
        'גני שרונה', 'הצפון החדש סביבת ככר המדינה', 'כוכב הצפון', 'אזור שדה דוב', 'צוקי אביב', 'גלילות', 
        'רמת אביב ג', 'תל ברוך', 'גני צהלה', 'הקריה', 'תל ברוך צפון', 'חוף הצוק', 'אזורי חן', 'רמת אביב', 'לינקולן'
    ],
    8: [
        'הצפון החדש החלק הדרומי', 'הצפון הישן החלק הצפוני', 'הצפון החדש החלק הצפוני', 'שיכון בבלי', 'תוכנית ל',
        'נופי ים', 'נוה אביבים', 'נאות אפקה א', 'נאות אפקה ב', 'רמת החייל', 'הצפון הישן החלק המרכזי',
        'הגוש הגדול', 'נמל תל אביב' 
    ],
    7: [
        'מונטיפיורי', 'נחלת יצחק', 'הצפון הישן החלק הדרום מערבי', 'הצפון הישן החלק הדרום מזרחי'
        ],
    6: [
        'הדר יוסף', 'בצרון', 'לב תל אביב החלק הצפוני', 'לב תל אביב החלק הדרומי', 'פלורנטין', 'בית שטראוס', 'רמת ישראל'
    ],
    5: [
        'רמת אביב החדשה', 'נוה שרת', 'כרם התימנים', 'תל חיים', 'רמת הטייסים', 'צפון יפו', 'גבעת הרצל', 'שבזי'
    ],
    4: [
        'יד אליהו', 'ניר אביב'
    ],
    3: [
        'נוה ברבור', 'נוה חן', 'כפיר'
    ],
    2: [
        'נוה שאנן', 'שפירא', 'קרית שלום', 'עזרא', 'לבנה', 'נוה אליעזר', 'שיכוני חסכון', 'דקר', 'צהלון', 'עג\'מי',
        'מרכז יפו מערבית לשדרות ירושלים', 'מרכז יפו מזרחית לשדרות ירושלים', 'תל כביר'
    ],
    1: [
        'יפו ג', 'יפו ד', 'התקווה'
]}

### Functions

In [ ]:
#פונקציה מהחלק הראשון של הפרויקט, למציאת מרחק בין שתי כתובות באמצעות google 
#היא משמשת כאן כפונקציית עזר לפונקציה אחרת שתהיה ב-Prepare Data
def get_distance(location1, location2):
    import requests
    import json
    url = f"https://routes.googleapis.com/directions/v2:computeRoutes"

    headers = {
        'Content-Type': 'application/json',
        'X-Goog-Api-Key': "AIzaSyAM5c_mZAF5n8B-////",
        'X-Goog-FieldMask': 'routes.distanceMeters'
    }

    body = {
        "origin": {
            "address": location1
        },
        "destination": {
            "address": location2
        },
        "travelMode": "DRIVE",  # BICYCLE / WALK / TWO_WHEELER / TRANSIT
        "routingPreference": "TRAFFIC_AWARE"
    }
    response = requests.post(url, headers=headers, data=json.dumps(body))

    try:
        if response.status_code == 200:
            try:
                data = response.json()
                distance_meters = data['routes'][0]['distanceMeters']
                return distance_meters
            except:
                #print("Response not in valid JSON format", response.text)
                pass
        else:
            print("Error:", response.status_code, response.text)
            return None
    except:
        print("Something went wrong with requests.get")



def compute_distances(df):
    distances = []

    for idx, row in df.iterrows():
        if pd.notna(row["address"]):
            full_address = f"{row['address']}, תל אביב"
        elif pd.notna(row["neighborhood"]):
            full_address = f"{row['neighborhood']}, תל אביב"
        else:
            distances.append(None)
            continue

        result = get_distance("כיכר דיזינגוף, תל אביב", full_address)
        if result:
            distances.append(result / 1000)  # Convert to km
        else:
            distances.append(None)

    return pd.Series(distances, index=df.index)


In [9]:
def remove_duplicates_by_columns(df):
    """
    מסיר שורות כפולות מה-DataFrame לפי שילוב של עמודות 'neighborhood', 'address', ו-'description'.
    שומר את ההופעה הראשונה בלבד של כל שילוב כזה, ומאפס את האינדקס לאחר מכן.
    """
    df.drop_duplicates(subset=['neighborhood', 'address', 'description'], keep='first', inplace=True)
    df.reset_index(drop=True, inplace=True)

In [ ]:
def normalize_property_type(df):
    """
    מנקה ומאחדת ערכים בעמודת 'property_type' לפי מיפוי מוגדר מראש,
    ומסווגת כל ערך שלא זוהה כ-'דירה'.
    """
    mapping = {
        'גג|פנטהאוז': 'גג/פנטהאוז',
        'גן|גינה': 'דירת גן',
        'דופלקס|דו': 'דופלקס',
        'סטודיו': 'סטודיו/לופט',
        'פרטי|קוטג\'|קוטג': 'פרטי/קוטג\'',
        'מרתף|פרטר': 'מרתף/פרטר',
        'יחידה|יחידת': 'יחידת דיור',
        'חניה': 'חניה',
        'מחסן': 'מחסן',
        'משרד': 'משרד',
        'חנות|מסחר': 'חנות',
        'אחר': 'אחר'
    }

    df.dropna(subset=['property_type'], inplace=True)

    df['property_type'] = df['property_type'].astype(str)

    for pattern, new_value in mapping.items():
        df.loc[df['property_type'].str.contains(pattern, case=False, na=False), 'property_type'] = new_value

    df.loc[~df['property_type'].isin(mapping.values()), 'property_type'] = 'דירה'

In [11]:
def filter_residential_properties(df):
    """
    שומרת רק שורות עם סוג נכס שהוא מגורים (לפי רשימה מוגדרת).
    """
    residential_types = ['דירה','גג/פנטהאוז','דירת גן','דופלקס','סטודיו/לופט','פרטי/קוטג\'','מרתף/פרטר','יחידת דיור']
    # הורדת השורות שאינן דירת מגורים
    df.drop(index=df[~df['property_type'].isin(residential_types)].index, inplace=True)

In [ ]:
def add_socioeconomic_rank(df, neighborhood_col='neighborhood', new_col='neighborhood_se'):
    """
    מוסיפה לדאטה עמודה עם דירוג סוציו־אקונומי לפי שכונה.
    הפונקציה משנה את df המקורי ישירות (in-place).
    
    פרמטרים:
        df (DataFrame): טבלת הנתונים לעדכון.
        neighborhood_col (str): שם העמודה שבה נמצאת השכונה.
        new_col (str): שם העמודה החדשה שתיווסף עם הדירוג.
    """

    neighborhood_se_rank = {
        neighborhood: rank
        for rank, neighborhoods in se_groups.items()
        for neighborhood in neighborhoods
    }


    df[new_col] = df[neighborhood_col].map(neighborhood_se_rank)

In [13]:
def add_district(df):
    """
    מוסיפה לעמודת df עמודה חדשה בשם 'district' המבוססת על עמודת 'neighborhood'
    ועל פי המילון tel_aviv_districts.
    """
    def find_district(neighborhood):
        for district, neigh_list in tel_aviv_districts.items():
            if neighborhood in neigh_list:
                return district
        return None  
    
    df['district'] = df['neighborhood'].apply(find_district)

In [14]:
def drop_neighborhoods(df):
        """
        מסירה שורות עם שכונה שלא נמצאת ברשימת neighborhoods.
        """
        df.drop(index=df[~df['neighborhood'].isin(neighborhoods)].index, inplace=True)

In [ ]:
def remove_small_area(df):
    """
    ממירה את העמודה 'area' למספרים, ומסירה שורות שבהן השטח קטן מ-15 מ"ר.
    """
    df['area'] = pd.to_numeric(df['area'], errors='coerce')
    df.drop(df[df['area'] <= 15].index, inplace=True)

In [16]:
def fix_ground_floor(df):
    """
    הפונקציה מחליפה ערכים חסרים ב-0 ולאחר מכן מחליפה בין העמודות "floor" ל-"total_floors"
    נגרם מטעות בכריית הנתונים
    """
    mask = (df['total_floors'].isnull()) & (df['neighborhood'].isin(['שיכון בבלי', 'הצפון החדש סביבת ככר המדינה']))
    df.loc[mask & df['floor'].isnull(), 'floor'] = 0
    df.loc[mask, 'total_floors'] = 0
    df.loc[mask, ['floor', 'total_floors']] = df.loc[mask, ['total_floors', 'floor']].values

In [ ]:

def fix_string_floor(df):
    """
    מתרגמת ערכים טקסטואליים בעמודת 'floor' לפורמט מספרי.
    """

    mask_ground = df["floor"].astype(str).str.contains("קרקע", na=False)
    df.loc[mask_ground, "floor"] = (
        df.loc[mask_ground, "floor"]
        .astype(str)
        .str.replace("קרקע", "0", regex=False)
    )

    mask = df["floor"].astype(str).str.contains(" מתוך ", na=False)

    df.loc[mask, "total_floors"] = (
        df.loc[mask, "floor"]
        .astype(str)
        .str.split(" מתוך ")
        .str[1]
    )

    df.loc[mask, "floor"] = (
        df.loc[mask, "floor"]
        .astype(str)
        .str.split(" מתוך ")
        .str[0]
    )

    df['floor'] = pd.to_numeric(df['floor'], errors='coerce')
    df['total_floors'] = pd.to_numeric(df['total_floors'], errors='coerce')

def fix_floors_with_floor(df):
    """
    מתקן טעויות בהן 'floor' גדול מ-'total_floors'.
    """
    def fix_floors(row):
        tot = row['total_floors']
        flr = row['floor']
        flr_str = str(flr)
        tot_str = str(tot)
        
        if tot == 0:
            return pd.Series({'floor': 0, 'total_floors': flr})
        else:
            if flr_str.endswith(tot_str):
                new_floor_str = flr_str[:-len(tot_str)]
                new_floor = int(new_floor_str) if new_floor_str else 0
            else:
                new_floor = flr
            return pd.Series({'floor': new_floor, 'total_floors': tot})

    mask = df['floor'] > df['total_floors']
    
    df.loc[mask, ['floor', 'total_floors']] = df.loc[mask].apply(fix_floors, axis=1)

In [ ]:
def fill_missing_garden_area(df):
    """
    ממירה את העמודה garden_area לערכים נומריים, וממלאת ערכים חסרים ב־0.
    """
    df['garden_area'] = pd.to_numeric(df['garden_area'], errors='coerce')
    
    df['garden_area'].fillna(0.0, inplace=True)

In [19]:
def binarize_days_to_enter(df):
    """
    הופכת את הערכים בעמודה days_to_enter לבינאריים:
    0 אם הערך הוא בדיוק 0, אחרת 1 (כולל ערכים חסרים).
    """
    df['days_to_enter'] = df['days_to_enter'].apply(lambda x: 0 if x == 0 else 1)

In [20]:
def fix_num_of_payments(df):
    """
    ממירה ערכים חסרים או 0 בעמודה num_of_payments לערך 12.
    """
    df['num_of_payments'] = df['num_of_payments'].apply(lambda x: 12 if pd.isna(x) or x == 0 else x)

In [ ]:
def clean_values(df):
    """
    מנקה ערכים חריגים במספר עמודות לפי כללים מוגדרים:

    - monthly_arnona:
        * ערך 0 → NaN
        * ערכים > 4000 → NaN
    - room_num, building_tax, area:
        * ערך 0 → NaN
    """
    df.loc[df['monthly_arnona'] > 4000, 'monthly_arnona'] = np.nan

    cols_with_invalid_zeros = ['room_num', 'building_tax', 'area', 'monthly_arnona']
    for col in cols_with_invalid_zeros:
        df[col].replace(0, np.nan, inplace=True)


In [22]:
def fix_large_building_tax(df):
    """
    מתקנת ערכי building_tax שגדולים מ־7000 ומסתיימים בספרה 0 (הסרה של ספרת הסיום).
    """
    mask = (df['building_tax'] >= 7000) & (df['building_tax'].notna())

    def remove_trailing_zero(value):
        s = str(int(value))
        if s.endswith("0"):
            return float(s[:-1])
        return value

    df.loc[mask, 'building_tax'] = df.loc[mask, 'building_tax'].apply(remove_trailing_zero)

In [23]:
def fill_zero_tax_for_low_buildings(df):
    """
    ממלא ערכים חסרים בעמודה building_tax עבור בניינים בני קומה אחת או פחות.
    ההנחה היא שבבניינים כאלה אין תשלום ארנונה לבניין → נכניס 0.
    """
    mask_low_buildings = (df['total_floors'] <= 1) & (df['building_tax'].isna())
    df.loc[mask_low_buildings, 'building_tax'] = 0

In [24]:
def fill_missing_num_of_images(df):
    """
    ממלא ערכים חסרים בעמודה num_of_images בערך 0.
    """
    df['num_of_images'] = df['num_of_images'].fillna(0)

In [25]:
def convert_to_km(df):
    """
    ממירה את הערכים בעמודה 'distance_from_center' מ־מטרים לקילומטרים במידת הצורך:
    - אם הערך מעל 100 → מניחים שזה מטרים ומחלקים ב־1000
    - אחרת → מניחים שזה כבר בקילומטרים
    הפונקציה פועלת ישירות על df (in-place).
    """
    mask = df['distance_from_center'].notna() & (df['distance_from_center'] >= 100)
    df.loc[mask, 'distance_from_center'] = df.loc[mask, 'distance_from_center'] / 1000

    

In [27]:
def fix_distance_km(df, compute_distances):
    """
    מזהה ערכים חריגים/חסרים בעמודת distance_from_center לפי קריטריונים:
    1. ערכים >12
    2. ערכים == 0
    3. הערך הכי רחוק מהממוצע בשכונה עם std>2
    4. ערכים חסרים
    ממלא מחדש רק עם compute_distances. המילוי הסופי (חציון שכונה) מתבצע בשלב מאוחר יותר.
    """
    def mark_outliers(df):
        cond1 = df['distance_from_center'] > 12
        cond2 = df['distance_from_center'] == 0

        # סטיית תקן רק על ערכים תקינים
        valid = df[(df['distance_from_center'] > 0) & (df['distance_from_center'] <= 12)]
        stds = valid.groupby('neighborhood')['distance_from_center'].std()
        high_std_neighs = stds[stds > 2].index

        cond3 = pd.Series(False, index=df.index)
        for neigh in high_std_neighs:
            group = valid[valid['neighborhood'] == neigh]
            if not group.empty:
                mean = group['distance_from_center'].mean()
                idx = (df['neighborhood'] == neigh) & ((df['distance_from_center'] - mean).abs() == (group['distance_from_center'] - mean).abs().max())
                cond3 = cond3 | idx

        return cond1 | cond2 | cond3

    # שלב 1: סימון חריגים וערכים חסרים
    mask_outliers = mark_outliers(df) | df['distance_from_center'].isna()

    # שלב 2: חישוב ערכים חדשים לחריגים/חסרים
    if mask_outliers.any():
        df.loc[mask_outliers, 'distance_from_center'] = compute_distances(df[mask_outliers])

In [28]:
def add_floor_ratio(df):
    """
    מוסיפה את הפיצ'ר 'floor_ratio' ל-DataFrame.
    חישוב: floor / total_floors.
    לוגיקה מעודכנת:
    - אם total_floors הוא 0 וגם floor הוא 0, הפיצ'ר יהיה 1.
    - אם total_floors הוא 0 ו-floor אינו 0, הפיצ'ר יהיה NaN.
    - אם floor או total_floors הם NaN, הפיצ'ר יהיה NaN.
    """

    def calculate_floor_ratio(row):
        floor = row['floor']
        total_floors = row['total_floors']

        # אם אחד מהם חסר, התוצאה היא NaN
        if pd.isna(floor) or pd.isna(total_floors):
            return np.nan
        
        # טיפול במקרה של 0/0
        if total_floors == 0 and floor == 0:
            return 1.0
        
        # טיפול במקרה של חלוקה באפס (רק total_floors הוא 0)
        if total_floors == 0:
            return np.nan
        
        # חישוב רגיל
        return floor / total_floors

    df['floor_ratio'] = df.apply(calculate_floor_ratio, axis=1)



def add_room_density(df):
    """
    מוסיפה את הפיצ'ר 'room_density' ל-DataFrame.
    חישוב: area / room_num.
    לוגיקה מעודכנת:
    - אם area או room_num הם NaN, הפיצ'ר יהיה NaN.
    - אם room_num הוא 0 (על פי בקשתך, לא צפוי לקרות אך עדיין מבוצעת בדיקה), הפיצ'ר יהיה NaN.
    """

    def calculate_room_density(row):
        area = row['area']
        room_num = row['room_num']

        # אם אחד מהם חסר, התוצאה היא NaN
        if pd.isna(area) or pd.isna(room_num):
            return np.nan
        
        # אם מספר החדרים הוא 0 (למרות שאמרת שלא יקרה), נחזיר NaN
        if room_num == 0:
            return np.nan
        
        # חישוב רגיל
        return area / room_num

    df['room_density'] = df.apply(calculate_room_density, axis=1)


In [29]:
def add_feature_interactions(df: pd.DataFrame):
    """
    מוסיפה אינטראקציות בין פיצ'רים:
    - area_neighborhood_interaction: area * neighborhood_se
    - parking_distance_interaction: has_parking * distance_from_center
    - balcony_floor_interaction: has_balcony * floor
    """
    df['area_neighborhood_interaction'] = df['area'] * df['neighborhood_se']
    df['parking_distance_interaction'] = df['has_parking'] * df['distance_from_center']
    df['balcony_floor_interaction'] = df['has_balcony'] * df['floor']

In [30]:
def drop_columns(df):
    """
    מסירה את העמודה 'address' מהדאטה פריים.
    """
    df.drop(columns=['address','description'], inplace=True)

## Prepare Data Function

In [ ]:
def PrepareData(data):
    """
    ניקוי דומייני של נתוני דירות
    פונקציה זו משתמשת בכל הפונקציות שהוגדרו למעלה ואיננה כוללת השלמות מבוססות סטטיסטיקה ומניפולציות מבוססת על עמודת מחיר
    """
    df = data.copy()

    remove_duplicates_by_columns(df)
    normalize_property_type(df)
    filter_residential_properties(df)
    add_district(df)
    add_socioeconomic_rank(df)
    drop_neighborhoods(df)
    remove_small_area(df)
    fix_ground_floor(df)
    fix_string_floor(df)
    fix_floors_with_floor(df)
    fill_missing_garden_area(df)
    fix_num_of_payments(df)
    binarize_days_to_enter(df)
    fix_large_building_tax(df)
    fill_zero_tax_for_low_buildings(df)
    clean_values(df)
    fill_missing_num_of_images(df)
    convert_to_km(df)
    fix_distance_km(df, compute_distances)
    drop_columns(df)
    add_floor_ratio(df)
    add_room_density(df)
    add_feature_interactions(df)

    return df

# PART 2

## Smart Imputer Class

<div dir="rtl">
מחלקה זו תוכננה להשתלב בפייפלין, במטרה למנוע זליגת נתונים בין קבוצות האימון והבחינה 
<br/>
תפקידה הוא השלמת הערכים החסרים על ידי שימוש בסטטיסטיקות שנלמדות מנתוני האימון. ביצענו בדיקות של מודלים שונים להשלמת העמודות בהם היה אחוז גובה של ערכים חסרים.
<br/>
<br/>
היא כוללת:
<br/>
1. השלמת ערכים חסרים בעמודת הארנונה על ידי מודל לינארי המבוסס על מספר חדרים, שטח ודירוג סוציואקונומי של השכונה
<br/>
2. השלמת ערכים חסרים בעמודת ועד בית על ידי מודל KNN המבוסס על הפיצ'רים המשפיעים על ועד הבית בבניין
<br/>
3. השלמת ערכים חסרים בעמודת שטח לפי השטח החציוני המתאים למספר החדרים בדירה
<br/>
4. השלמת ערכים חסרים בעמודת מספר חדרים לפי מודל KNN המתבסס על שטח הדירה
<br/>
5. השלמת ערכים חסרים בעמודת מרחק מהמרכז לפי המרחק החציוני השכונתי. הערכים החריגים נקבעו לפי בדיקה של מרחק סביר מהמרכז לשאר שכונות תל אביב

In [35]:
class SmartImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.imputer_arnona = None
        self.imputer_tax = None
        self.imputer_room_num = None 
        self.median_area_by_rooms = {}
        self.distance_median_by_neighborhood = {}
        self.model_arnona = None
        self.floor_ratio_median = None
        self.room_density_median = None
        self.median_fill_columns = ['floor','total_floors','room_density',
                                    'floor_ratio', 'balcony_floor_interaction',
                                    'area_neighborhood_interaction','parking_distance_interaction']
        self.medians = {}

    def fit(self, X, y=None):
        self._fit_arnona(X)
        X_temp = self._transform_arnona(X.copy())
        self._fit_tax(X_temp)
        self._fit_area(X)
        X_temp_area = self._transform_area(X.copy())
        self._fit_room_num(X_temp_area)
        self._fit_distance(X)
        for col in self.median_fill_columns:
            if col in X.columns:
                self.medians[col] = X[col].median()
        
        return self


    def _fit_arnona(self, X):
        X_clean = X.dropna(subset=['room_num', 'area', 'monthly_arnona','neighborhood_se'])  
        X_train = X_clean[['room_num', 'area', 'neighborhood_se']]  
        y_train = X_clean['monthly_arnona']      
        self.model_arnona = LinearRegression()
        self.model_arnona.fit(X_train, y_train) 

    def _fit_tax(self, X):
        cols = ['total_floors', 'elevator', 'monthly_arnona', 'has_parking', 'building_tax','neighborhood_se']
        df = X[cols]
        self.scaler_tax = StandardScaler()
        scaled_df = self.scaler_tax.fit_transform(df)
        self.imputer_tax = KNNImputer(n_neighbors=5)
        self.imputer_tax.fit(scaled_df)


    def _fit_area(self, X):
        self.median_area_by_rooms = (
            X.groupby('room_num')['area']
            .median()
            .to_dict()
        )

    def _fit_room_num(self, X):
        cols = ['area', 'room_num']
        df = X[cols]
        self.scaler_room_num = StandardScaler()
        scaled_df = self.scaler_room_num.fit_transform(df)
        self.imputer_room_num = KNNImputer(n_neighbors=8)
        self.imputer_room_num.fit(scaled_df)
    

    def _fit_distance(self, X):
        # חישוב חציון distance_km לפי שכונה על ערכים תקינים בלבד
        valid = X[(X['distance_from_center'] > 0) & (X['distance_from_center'] <= 12)]
        self.distance_median_by_neighborhood = (
            valid.groupby('neighborhood')['distance_from_center']
            .median()
            .to_dict()
        )

    def transform(self, X):
        X_copy = X.copy()
        
        X_copy = self._transform_area(X_copy)
        X_copy = self._transform_room_num(X_copy) 
        X_copy = self._transform_arnona(X_copy) 
        X_copy = self._transform_tax(X_copy)     
        X_copy = self._transform_distance(X_copy)
        for col in self.median_fill_columns:
            if col in X_copy.columns and col in self.medians:
                X_copy[col].fillna(self.medians[col], inplace=True)

        return X_copy
    
    def _transform_arnona(self, X):
        mask = X['monthly_arnona'].isna()
        if mask.any():
            # רק שורות שיש בהן גם room_num וגם area לא חסרים
            valid_mask = mask & X['room_num'].notna() & X['area'] & X['neighborhood_se'].notna() 
            missing_mask = mask & (~valid_mask)
            # חיזוי עם המודל
            if valid_mask.any():
                X_missing_arnona = X.loc[valid_mask, ['room_num', 'area','neighborhood_se']]
                predicted_arnona = self.model_arnona.predict(X_missing_arnona)
                X.loc[valid_mask, 'monthly_arnona'] = predicted_arnona
            # מילוי חציון לשאר
            n_missing = missing_mask.sum()
            if n_missing > 0:
                median_arnona = X['monthly_arnona'].median()
                X.loc[missing_mask, 'monthly_arnona'] = median_arnona        
        return X

    def _transform_tax(self, X):
        cols = ['total_floors', 'elevator', 'monthly_arnona', 'has_parking', 'building_tax','neighborhood_se']
        df = X[cols]
        scaled_df = self.scaler_tax.transform(df)
        imputed = self.imputer_tax.transform(scaled_df)
        X['building_tax'] = imputed[:, -1]
        return X
    
    def _transform_area(self, X):
        mask = X['area'].isna()  
        mask = X['area'].isna()
        X.loc[mask, 'area'] = X.loc[mask, 'room_num'].map(self.median_area_by_rooms)
        return X
    
    def _transform_room_num(self, X):
        cols = ['area', 'room_num']
        df = X[cols]
        scaled_df = self.scaler_room_num.transform(df)
        imputed = self.imputer_room_num.transform(scaled_df)
        X['room_num'] = imputed[:, 1]
        return X

       
    def _transform_distance(self, X):
        """
        ממלא ערכים חריגים או חסרים ב-'distance_from_center' לפי חציון שכונתי
        שנלמד במהלך fit().
        """
        def mark_outliers(df):
            cond1 = df['distance_from_center'] > 12
            cond2 = df['distance_from_center'] == 0

            valid = df[(df['distance_from_center'] > 0) & (df['distance_from_center'] <= 12)]
            stds = valid.groupby('neighborhood')['distance_from_center'].std()
            high_std_neighs = stds[stds > 2].index

            cond3 = pd.Series(False, index=df.index)
            for neigh in high_std_neighs:
                group = valid[valid['neighborhood'] == neigh]
                if not group.empty:
                    mean = group['distance_from_center'].mean()
                    idx = (df['neighborhood'] == neigh) & ((df['distance_from_center'] - mean).abs() == (group['distance_from_center'] - mean).abs().max())
                    cond3 = cond3 | idx

            return cond1 | cond2 | cond3

        mask_outliers = mark_outliers(X) | X['distance_from_center'].isna()

        for neigh in X.loc[mask_outliers, 'neighborhood'].unique():
            median_val = self.distance_median_by_neighborhood.get(neigh, X['distance_from_center'].median())
            X.loc[(X['neighborhood'] == neigh) & mask_outliers, 'distance_from_center'] = median_val

        return X
    


## ElasticNet Model

### Encoding & Scaling

In [36]:
from sklearn.pipeline import make_pipeline

numeric_features = [
    'room_num', 'floor', 'area',
    'monthly_arnona', 'building_tax',
     'total_floors', 'num_of_images',
    'distance_from_center','neighborhood_se','room_density', 'floor_ratio',
    'balcony_floor_interaction', 'area_neighborhood_interaction',
    'parking_distance_interaction'
]

full_preprocessor = ColumnTransformer([
    ('neigh_target_scaler', make_pipeline(TargetEncoder(), StandardScaler()), ['neighborhood']),
    ('onehot', OneHotEncoder(handle_unknown='ignore'), ['property_type','district']),
    ('num', StandardScaler(), numeric_features)
], remainder='passthrough')

### Pipeline

<div dir="rtl">
הדאטה מגיע לשלב זה לאחר הסרת השורות וההכנה הראשונה. כאן הוא עובר:
<br/>
- השלמות שהגדרנו במחלקת SmartImputer
<br/>
- העמודה "שכונה" מקודדת באמצעות target encoder לפי ממוצע המחירים בשכונה 
<br/>
בחרנו להשתמש בקידוד זה כיון שיש יחסית הרבה שכונות, ורצינו להפוך את העמודה לעמודה מספרית אינפורמטיבית 
<br/>
- העמודה "סוג הנכס" מקודדת באמצעות one hot encoder
<br/>
- כל העמודות הנומריות עוברות נירמול

In [37]:
pipeline = Pipeline(steps=[
    ('impute', SmartImputer()),
    ('preprocess', full_preprocessor),
    ('model', ElasticNet())
])

### Model

In [38]:
# הכנס את price לעמודות, נקה הכל, ואז תפריד
data_clean = PrepareData(data)
X = data_clean.drop(columns=['price'])
y = data_clean['price']

In [38]:
#בחרנו להוריד מראש את העמודת הלוו כיוון שהשונות שלהן קטנה מאד
X.drop(columns=['garden_area', 'num_of_payments' ],inplace=True)

In [39]:
kf = KFold(n_splits=10, shuffle=True, random_state=42) 

# הגדרת היפרפרמטרים עבור מודל זה 
param_distributions = {
    'model__alpha': uniform(0.01, 10),  
    'model__l1_ratio': uniform(0, 1)   
}

# RandomizedSearchCV object
random_search = RandomizedSearchCV(
    pipeline,                
    param_distributions,     
    n_iter=50,             
    cv=kf,                   
    scoring='r2',         
    n_jobs=-1,            
    random_state=42        
)

random_search.fit(X, y)

pipeline.set_params(**random_search.best_params_)

Pipeline(steps=[('impute', SmartImputer()),
                ('preprocess',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('neigh_target_scaler',
                                                  Pipeline(steps=[('targetencoder',
                                                                   TargetEncoder()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['neighborhood']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['property_type',
                                                   'district']),
                                                 ('num', StandardScaler(),
                       

In [40]:
#בדיקת ביצועי המודל באמצעות fold cross-validation

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)


scores_rmse = cross_val_score(
    pipeline, X, y,
    cv=kf,
    scoring=rmse_scorer
)

scores_mae = cross_val_score(
    pipeline, X, y,
    cv=kf,
    scoring='neg_mean_absolute_error'
)


r2_scores = cross_val_score(
    pipeline, X, y,
    cv=kf,
    scoring='r2'
)


mean_rmse = round(scores_rmse.mean(), 2)
mean_mae = round(-scores_mae.mean(), 2)
mean_r2 = round(r2_scores.mean(), 2)

print("תוצאות RMSE ב־10 קיפולים:")
print(np.round(scores_rmse, 2))
print(" ממוצע RMSE:", mean_rmse)

print("\nתוצאות MAE ב־10 קיפולים:")
print(np.round(-scores_mae, 2))
print(" ממוצע MAE:", mean_mae)

print("\nתוצאות R^2 ב־10 קיפולים:")
print(np.round(r2_scores, 2))
print(" ממוצע R^2:", mean_r2)

תוצאות RMSE ב־10 קיפולים:
[2460.88 2702.36 3860.54 2849.98 3410.15 2556.12 2869.   2522.29 3708.59
 2730.14]
 ממוצע RMSE: 2967.01

תוצאות MAE ב־10 קיפולים:
[1615.44 1697.47 2035.32 1740.29 1820.78 1900.51 1719.18 1780.41 1995.77
 1667.06]
 ממוצע MAE: 1797.22

תוצאות R^2 ב־10 קיפולים:
[ 0.66  0.72 -0.05  0.63  0.46  0.6   0.73  0.72  0.67  0.78]
 ממוצע R^2: 0.59


### Saving the Model

In [42]:
import joblib

pipeline.fit(X, y)
joblib.dump(pipeline, "trained_ElasticNet_pipeline.joblib")

['trained_ElasticNet_pipeline.joblib']

### Model Results Analysis

In [41]:
# פונקציה לקבלת קיפול מסוים לטובת ניתוח הקיפולים
def get_fold_data_with_predictions(X, y, pipeline, fold_number=0, kf=None):
    folds = list(kf.split(X, y))
    train_idx, test_idx = folds[fold_number]
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    df_fold = X_test.copy()
    df_fold['y_true'] = y_test
    df_fold['y_pred'] = y_pred
    df_fold['res'] = df_fold['y_true'] - df_fold['y_pred']
    df_fold['absolute_res'] = df_fold['res'].abs()
    return df_fold


# fold = get_fold_data_with_predictions(X, y, pipeline, fold_number=4, kf=kf)
# fold.sort_values(by='absolute_res', ascending=False)

In [41]:
def get_top_misses(X, y, pipeline, top_n=10):
    """
    מחזירה את top_n הדוגמאות עם הפספוס הכי גדול (בערך מוחלט) על כל הדאטה.
    """
    pipeline.fit(X, y)
    y_pred = pipeline.predict(X)
    df = X.copy()
    df['y_true'] = y
    df['y_pred'] = y_pred
    df['miss'] = (df['y_true'] - df['y_pred']).abs()
    return df.sort_values('miss', ascending=False).head(top_n)

top_misses = get_top_misses(X, y, pipeline, top_n=20)
top_misses

,property_type,neighborhood,room_num,floor,area,days_to_enter,monthly_arnona,building_tax,total_floors,has_parking,...,district,neighborhood_se,floor_ratio,room_density,area_neighborhood_interaction,parking_distance_interaction,balcony_floor_interaction,y_true,y_pred,miss
701,דירה,שיכון בבלי,5.0,20.0,329.0,0,NaN,2700.0,20.0,1,...,רובע 4,8.0,1.000000,65.800000,2632.0,2.897,20.0,6000.0,25998.838784,19998.838784
655,דירה,שבזי,4.0,33.0,315.0,0,NaN,NaN,20.0,0,...,רובע 5,5.0,1.650000,78.750000,1575.0,0.000,33.0,45000.0,25927.875922,19072.124078
152,דירה,לב תל אביב החלק הדרומי,4.0,0.0,315.0,1,NaN,700.0,34.0,1,...,רובע 5,6.0,0.000000,78.750000,1890.0,3.406,0.0,45000.0,25973.657912,19026.342088
721,דירת גן,הצפון הישן החלק הדרום מערבי,4.0,0.0,200.0,0,NaN,680.0,7.0,1,...,רובע 3,7.0,0.000000,50.000000,1400.0,1.990,0.0,37000.0,18012.704305,18987.295695
728,דירה,הצפון הישן החלק הדרום מערבי,3.0,10.0,230.0,0,NaN,NaN,15.0,1,...,רובע 3,7.0,0.666667,76.666667,1610.0,0.860,10.0,38000.0,22439.343600,15560.656400
326,דירה,לב תל אביב החלק הצפוני,3.0,8.0,180.0,0,NaN,2900.0,10.0,0,...,רובע 5,6.0,0.800000,60.000000,1080.0,0.000,8.0,30000.0,16244.765629,13755.234371
499,פרטי/קוטג',הצפון החדש החלק הדרומי,8.0,0.0,450.0,1,NaN,NaN,0.0,1,...,רובע 4,8.0,1.000000,56.250000,3600.0,2.286,0.0,45000.0,31820.521598,13179.478402
495,דירה,הצפון החדש החלק הדרומי,4.0,1.0,180.0,0,NaN,1500.0,8.0,1,...,רובע 4,8.0,0.125000,45.000000,1440.0,2.286,1.0,30000.0,17126.389425,12873.610575
576,גג/פנטהאוז,הצפון הישן החלק הדרום מזרחי,5.0,5.0,160.0,1,NaN,600.0,5.0,1,...,רובע 3,7.0,1.000000,32.000000,1120.0,1.400,5.0,26500.0,14431.367699,12068.632301
333,דירה,לב תל אביב החלק הצפוני,NaN,9.0,147.0,0,NaN,NaN,20.0,1,...,רובע 5,6.0,0.450000,NaN,882.0,3.695,9.0,25000.0,13225.694371,11774.305629


<div dir="rtl">
בעת הערכת ביצועי המודל באמצעות Cross-Validation של 10 קיפולים, התקבלה שונות גבוהה יחסית בין תוצאות הקיפולים אחת הסיבות האפשריות לכך היא נוכחות של ערכים קיצוניים אמיתיים בדאטה (וידאנו שאין data leakage) – למשל דירות יוקרה חריגות במחיר או נכסים נדירים – שהתפזרו באופן לא שווה בין הקיפולים והשפיעו בצורה לא סימטרית על ביצועי המודל.
<br/>
בנוסף, גודל הדאטה הכולל עומד על כ־750 שורות בלבד, כך שכל קיפול כולל פחות מ־700 דוגמאות לאימון וכ־75 דוגמאות לבדיקה. מדובר סט בדיקה קטן יחסית, מה שמעלה את הרגישות של כל קיפול להרכב הספציפי של הדאטה בסט הבדקיה, ועלול להגביר את השונות במדדים. ראיה לסברה לזו היא שהרצנו על פחות קיפולים או על 80% מהדאטה (מקרים פחות רגישים לחריגים) ראינו ביצועים יציבים יותר. 
<br/>
דבר נוסף, ראינו שבחלק מהמקומות שהמודל פספס בצורה חריגה גם אנחנו בכבני אדם היינו יותר קרובים להערכה שלו מאשר למחיר שבדאטה. פער זה נובע מהנתונים שעלי לאתר וכנראה אנשים החליטו להעלות מחירים נמוכים יותר לדירות שלהם בשביל לא ליפול בפלטורים של מחפשי דירות באתר או ממניעים שונים. לדוגמא דירת פנזטהואז בשיכון בבלי במגדל בשטח 320 מטר רק ב6000 שקל (והמודל חזה מחיר גבוה בהרבה ,ובצדק)
<br/>

**בשורה תחתונה:**  
ניתן להסביר את השונות ואת הביצועים נמוכים יחסית הן בגלל הקיפולים ("בעיה" שאמור להיפתר בהרצה על כל הדאטה)  
והן בגלל הדאטה עצמו ובעיקר עמודת המחיר

### Features Table

In [46]:
from sklearn.inspection import permutation_importance

def get_feature_importance(pipeline, X, y):
    """
    מחשבת חשיבות פיצ'רים עבור מודל באמצעות מקדמים ו-permutation importance
    
    Args:
        pipeline: Pipeline מאומן
        X: דאטה לחישוב חשיבות הפיצ'רים
        y: ערכי המטרה
    
    Returns:
        DataFrame עם חשיבות הפיצ'רים ממוין לפי חשיבות יחסית
    """
    # הכנת הדאטה
    X_processed = pipeline.named_steps['preprocess'].transform(
        pipeline.named_steps['impute'].transform(X)
    )
    feature_names = pipeline.named_steps['preprocess'].get_feature_names_out()
    
    # חישוב מקדמים וחשיבות יחסית
    coefs = pipeline.named_steps['model'].coef_
    coef_df = pd.DataFrame({
        'feature': feature_names,
        'coefficient': coefs,
        'abs_coefficient': np.abs(coefs)
    })
    coef_df['relative_importance'] = coef_df['abs_coefficient'] / coef_df['abs_coefficient'].sum()
    
    # חישוב permutation importance
    perm_result = permutation_importance(
        pipeline.named_steps['model'], 
        X_processed, y,
        n_repeats=30, 
        random_state=42, 
        scoring='r2'
    )
    coef_df['permutation_importance_mean'] = perm_result.importances_mean
    
    # מיון לפי חשיבות יחסית
    return coef_df.sort_values('relative_importance', ascending=False).reset_index(drop=True)

# שימוש בפונקציה
pd.set_option('display.float_format', lambda x: f'{x:.4f}')
feature_importance_df = get_feature_importance(pipeline, X, y)
feature_importance_df


,feature,coefficient,abs_coefficient,relative_importance,permutation_importance_mean
0,num__area,1324.1591,1324.1591,0.0924,0.1526
1,onehot__district_רובע 5,1153.2354,1153.2354,0.0805,0.0210
2,num__area_neighborhood_interaction,1146.6132,1146.6132,0.0800,0.1142
3,num__room_density,835.1622,835.1622,0.0583,0.0611
4,onehot__district_רובע 9,-691.3708,691.3708,0.0483,0.0061
5,remainder__is_furnished,688.9732,688.9732,0.0481,0.0100
6,onehot__district_רובע 3,674.7181,674.7181,0.0471,0.0084
7,onehot__district_רובע 4,-670.6678,670.6678,0.0468,0.0063
8,neigh_target_scaler__neighborhood,539.1939,539.1939,0.0376,0.0245
9,remainder__has_parking,527.7429,527.7429,0.0368,0.0071


## Decision Tree-Based Model

### Encoding

In [47]:
# שלב הקידוד – property_type, district → OneHot, neighborhood → Target
categorical_encoder2 = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), ['property_type', 'district']),
        ('target', TargetEncoder(), ['neighborhood'])
    ],
    remainder='passthrough'
)

### Pipeline

In [ ]:
pipeline_XGboost = Pipeline([
    ('smart_imputer', SmartImputer()),
    ('encoder', categorical_encoder2),
    ('model', XGBRegressor())
])


### Model

<div dir="rtl">
בחרנו לבדוק את ההיפר פרמטרים עם RandomizedSearchCV:  
<br/>
1. למודל יש הרבה פרמטרים וממילא יש הרבה קומבינציות אפשריות בין הפרמטרים
<br/>
2. הרבה מהפרמטרים הם פרמטרים רציפים
<br/>
3. לא היה לנו כיוון התחלתי לבדיקת היפר פרמטרים ולכן בדיקה של הרבה כיוונים במקביל הייתה יעילה יותר מבחינתנו

In [ ]:
# בדיקת פרמטרים עם RandomizedSearchCV
xgb_param_dist = {
    'model__n_estimators': randint(50, 300),
    'model__learning_rate': uniform(0.01, 0.3),
    'model__max_depth': randint(2, 10),
    'model__min_child_weight': randint(1, 6),
    'model__subsample': uniform(0.6, 0.4),
    'model__colsample_bytree': uniform(0.6, 0.4),
    'model__gamma': uniform(0, 5)
}

xgb_random_search = RandomizedSearchCV(
    pipeline_XGboost,
    param_distributions=xgb_param_dist,
    n_iter=20,
    scoring='neg_root_mean_squared_error',
    cv=kf,
    random_state=42,
    n_jobs=-1,
    verbose=1
)

xgb_random_search.fit(X, y)

print(" Best Parameters (RMSE):", xgb_random_search.best_params_)
print("Best RMSE Score:", round(-xgb_random_search.best_score_, 2)) 


Fitting 10 folds for each of 20 candidates, totalling 200 fits
 Best Parameters (RMSE): {'model__colsample_bytree': 0.9439761626945282, 'model__gamma': 3.4015376929388985, 'model__learning_rate': 0.1451497755908629, 'model__max_depth': 3, 'model__min_child_weight': 4, 'model__n_estimators': 138, 'model__subsample': 0.9862528132298237}
Best RMSE Score: 2858.68


<div dir="rtl">
בשביל שלב שני אחרי RandomizedSearchCV בשביל לבדוק כיוון עדין ומדויק יותר באזורים האלה

In [63]:
#בדיקת פרמטרים עם GridSearchCV
param_grid = {
    'model__n_estimators': [100, 150, 200],
    'model__max_depth': [3, 5, 8],
    'model__learning_rate': [0.1, 0.15, 0.2],
    'model__subsample': [0.8, 1.0]
}

# Grid Search לפי R²
grid_search_r2 = GridSearchCV(
    pipeline_XGboost,
    param_grid,
    cv=kf,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid_search_r2.fit(X, y)

# Grid Search לפי RMSE (צריך להפוך ל-negative כי sklearn ממקסם)
neg_rmse_scorer = make_scorer(mean_squared_error, squared=False, greater_is_better=False)

grid_search_rmse = GridSearchCV(
    pipeline_XGboost,
    param_grid,
    cv=kf,
    scoring=neg_rmse_scorer,
    n_jobs=-1,
    verbose=0  # בלי הדפסות חוזרות
)

grid_search_rmse.fit(X, y)

# תוצאות
print(" Best Parameters (R²):", grid_search_r2.best_params_)
print(" Best R² Score:", round(grid_search_r2.best_score_, 4))

print(" Best Parameters (RMSE):", grid_search_rmse.best_params_)
print("Best RMSE Score:", round(-grid_search_rmse.best_score_, 2)) 

Fitting 10 folds for each of 54 candidates, totalling 540 fits
 Best Parameters (R²): {'model__learning_rate': 0.15, 'model__max_depth': 3, 'model__n_estimators': 100, 'model__subsample': 1.0}
 Best R² Score: 0.6429
 Best Parameters (RMSE): {'model__learning_rate': 0.15, 'model__max_depth': 3, 'model__n_estimators': 150, 'model__subsample': 1.0}
Best RMSE Score: 2864.59


In [64]:
pipeline_XGboost.set_params(**grid_search_rmse.best_params_)

Pipeline(steps=[('smart_imputer', SmartImputer()),
                ('encoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['property_type',
                                                   'district']),
                                                 ('target', TargetEncoder(),
                                                  ['neighborhood'])])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=No...
                              feature_types=None, gamma=3.4015376929388985,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=0.15,
                       

In [ ]:
#בדיקת ביצועי המודל באמצעות fold cross-validation

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
kf = KFold(n_splits=10, shuffle=True, random_state=42) 

scores_rmse_XGboost = cross_val_score(
    pipeline_XGboost, X, y,
    cv=kf,
    scoring=rmse_scorer
)

scores_mae_XGboost = cross_val_score(
    pipeline_XGboost, X, y,
    cv=kf,
    scoring='neg_mean_absolute_error'
)


r2_scores_XGboost = cross_val_score(
    pipeline_XGboost, X, y,
    cv=kf,
    scoring='r2'
)


mean_rmse_XGboost = round(scores_rmse_XGboost.mean(), 2)
mean_mae_XGboost = round(-scores_mae_XGboost.mean(), 2)
mean_r2_XGboost = round(r2_scores_XGboost.mean(), 2)

print("תוצאות RMSE ב־10 קיפולים:")
print(np.round(scores_rmse_XGboost, 2))
print(" ממוצע RMSE:", mean_rmse_XGboost)

print("\nתוצאות MAE ב־10 קיפולים:")
print(np.round(-scores_mae_XGboost, 2))
print(" ממוצע MAE:", mean_mae_XGboost)

print("\nתוצאות R^2 ב־10 קיפולים:")
print(np.round(r2_scores_XGboost, 2))
print(" ממוצע R^2:", mean_r2_XGboost)

# PART 3

## Model Comparison

In [66]:
metrics_comparison = pd.DataFrame({
    'Model': ['ElasticNet', 'XGBoost'],
    'RMSE': [mean_rmse, mean_rmse_XGboost],
    'MAE': [mean_mae, mean_mae_XGboost],
    'R²': [mean_r2, mean_r2_XGboost]
})

metrics_comparison

,Model,RMSE,MAE,R²
0,ElasticNet,2967.0100,1797.2200,0.5900
1,XGBoost,2864.5900,1581.8200,0.6400


<div dir="rtl">
ניתן לראות שהמודל המבוסס על עצים (XGBoost) הציג ביצועים טובים יותר במטריקות ההערכה לעומת מודל ElasticNet. לדעתנו, פער זה נובע מכמה סיבות עיקריות:

1. למודלי עצים יש יכולת לזהות קשרים מורכבים ולא ליניאריים בין הפיצ'רים לבין ערך המטרה, בעוד שמודל ElasticNet מתאים בעיקר לקשרים ליניאריים. למרות שניסינו להוסיף פיצ'רים שמייצגים אינטראקציות בין משתנים, ייתכן שלא הצלחנו לבטא את המורכבות המלאה שקיימת בדאטה.

2. מודל XGBoost מסוגל להתמודד טוב יותר עם ערכים חסרים, פיצ'רים קטגוריים מקודדים, ופיצ'רים לא נורמליים – מבלי שנדרש לבצע עליהם טרנספורמציות מורכבות. זה מאפשר לו לנצל את המידע הגולמי בצורה אפקטיבית יותר.

3. ייתכן שבדאטה שלנו קיימות קבוצות תת־אוכלוסייה שמתנהגות שונה (למשל שכונות מסוימות או סוגי נכסים מסוימים), ומודל מבוסס עצים יודע "לחתוך" את הדאטה לקבוצות כאלה וללמוד מהן באופן נפרד – מה שלא מתאפשר באותו אופן במודל ליניארי.


In [67]:
# אחרי אימון הפייפליין
pipeline_XGboost.fit(X, y)

# קבלת המודל
xgb_model = pipeline_XGboost.named_steps['model']

# קבלת שמות הפיצ'רים מה-ColumnTransformer
encoder = pipeline_XGboost.named_steps['encoder']
feature_names = encoder.get_feature_names_out()

# חשיבות לפי gain
importance_gain = xgb_model.get_booster().get_score(importance_type='gain')

# יצירת DataFrame עם השמות הנכונים
feature_importance_xgboost_df = pd.DataFrame([
    {'feature': feature_names[int(k.replace('f', ''))], 'gain': v} 
    for k, v in importance_gain.items()
]).sort_values('gain', ascending=False)



In [68]:


# קח את חמשת הפיצ'רים המובילים מכל מודל
top5_coef = feature_importance_df.sort_values("coefficient", ascending=False).head(5).reset_index(drop=True)
top5_xgb = feature_importance_xgboost_df.sort_values("gain", ascending=False).head(5).reset_index(drop=True)

# צור DataFrame השוואתי
compare_df = pd.DataFrame({
    'ElasticNet_feature': top5_coef['feature'],
    'XGBoost_feature': top5_xgb['feature'],
})

compare_df


,ElasticNet_feature,XGBoost_feature
0,num__area,remainder__area
1,onehot__district_רובע 5,remainder__neighborhood_se
2,num__area_neighborhood_interaction,remainder__elevator
3,num__room_density,remainder__has_parking
4,remainder__is_furnished,remainder__room_density


<div dir="rtl">
ניתוח הפיצ'רים:
<br/>
- שני המודלים סימנו את שטח הדירה כפיצ'ר חשוב. שטח הדירה הוא גורם ישיר במחיר השכירות, ולכן באופן טבעי מודלים שונים מזהים אותו כמשתנה מרכזי.
<br/>
- room_density מופיע בשני המודלים  זהו מאפיין מהונדס שכנראה מהווה אינדיקטור חשוב ליעילות ניצול השטח בדירה, וסביר שמשפיע על תמחור (לדוגמה, דירה גדולה עם מעט חדרים יכולה להיתפס כמרווחת יותר). 
<br/>
- שני המודלים מזהים את חשיבות מיקום הנכס (שכונה): בעוד ש-Elastic Net משלב את השכונה באמצעות פיצ'ר מהונדס של אינטראקציה (area_neighborhood_interaction), מודל ה-XGBoost מתייחס ישירות לדירוג הסוציו-אקונומי (neighborhood_se), מה שמדגיש את יכולתו ללכוד השפעות עקיפות ומורכבות יותר של הסביבה.
<br/>
- XGBoost בחר מאפיינים בינאריים במיקום 3-4.  עצי החלטה יכולים לנצל בקלות פיצ'רים כאלה על ידי פיצול הנתונים לשני ענפים (עם מעלית / בלי מעלית), מה שמוביל לשיפור ניכר בחיזוי. מודלים לינאריים יכולים גם להשתמש בהם, אבל עצי החלטה יתפסו את חשיבותם בצורה חזקה יותר דרך סדרת הפיצולים.

<div dir="rtl">
לסיכום, ElasticNetבחר פיצ'רים שנוצרו בצורה ידנית או לאחר טרנספורמציה. זה הגיוני, כיוון מודלים ליניאריים כמו ElasticNet לא מסוגלים לתפוס אינטראקציות מורכבות בעצמם, ולכן יש צורך להנדס מראש פיצ'רים שמייצגים קשרים לא ישירים.  
<br/>
XGBoost בחר גם פיצ'רים בינאריים בסיסיים, הוא יכול למצוא קשרים מורכבים בעצמו, ולכן נשען גם על פיצ'רים גולמיים ובינאריים בלי הנדסה מרובה.  